In [2]:
from itertools import product

import numpy as np
import xarray as xr
from scipy import integrate
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pop_tools
import xpersist as xp
import dask


import util
plt.style.use('ggplot')

In [3]:
variables = ['SST', 'Chl_surf', 'KGP']

ds = xr.open_zarr('data/cesm-le-djf-triregion-timeseries.zarr')
ds = ds[variables].sel(length = 40)
ds

<xarray.Dataset>
Dimensions:    (member_id: 34, region: 5, time: 180)
Coordinates:
  * time       (time) int64 1921 1922 1923 1924 1925 ... 2097 2098 2099 2100
  * region     (region) <U17 'Southern Ocean' 'WAP & Atlantic' ... 'AP Sector'
  * member_id  (member_id) int64 1 2 9 10 11 12 13 ... 34 35 101 102 103 104 105
    length     float64 40.0
Data variables:
    SST        (time, member_id, region) float64 dask.array<chunksize=(180, 34, 5), meta=np.ndarray>
    Chl_surf   (time, member_id, region) float64 dask.array<chunksize=(180, 34, 5), meta=np.ndarray>
    KGP        (time, member_id, region) float64 dask.array<chunksize=(90, 34, 5), meta=np.ndarray>

In [ ]:
def assess_toe(da):
    """ compute toe on the basis of henson et al 2017"""
    # compute anomalies
    da_ref = da.sel(time=slice(1920, 1950)).mean(['time'])
    anom = da - da_ref
    
    # find inflection point
    time = da.time.values
    anom_cumtrap = integrate.cumtrapz(anom.values, time, initial=0)
    
    for l in range(len(time)):
        if all(anom_cumtrap[l:] > 0): #decline
            ndx = l
            break
    
    beta = np.polyfit(time[ndx:], anom[ndx:], 1)
    
    return time[ndx], beta[0]


In [4]:
#assess_toe(ds.KGP.sel(region='Southern Ocean').mean('member_id'))

In [5]:
assess_toe(ds.SST.sel(region='Southern Ocean').mean('member_id'))

(1950, 0.02023557793552905)

## Load DJF dataset

In [6]:
variables = ['SST', 'Chl_surf', 'KGP', 'NPP', 'Jint_100m_DIC']

ds_djf = xr.open_zarr(f'{util.project_tmpdir}/cesm-le-fields-djf.zarr')
ds_djf = ds_djf[variables].sel(length = 40)
ds_djf

<xarray.Dataset>
Dimensions:        (member_id: 34, nlat: 93, nlon: 320, time: 180)
Coordinates:
    ULONG          (nlat, nlon) float64 dask.array<chunksize=(93, 320), meta=np.ndarray>
  * time           (time) int64 1921 1922 1923 1924 1925 ... 2097 2098 2099 2100
    TLAT           (nlat, nlon) float64 dask.array<chunksize=(93, 320), meta=np.ndarray>
    TLONG          (nlat, nlon) float64 dask.array<chunksize=(93, 320), meta=np.ndarray>
    KMT            (nlat, nlon) float64 dask.array<chunksize=(93, 320), meta=np.ndarray>
    TAREA          (nlat, nlon) float64 dask.array<chunksize=(93, 320), meta=np.ndarray>
  * member_id      (member_id) int64 1 2 9 10 11 12 ... 35 101 102 103 104 105
    ULAT           (nlat, nlon) float64 dask.array<chunksize=(93, 320), meta=np.ndarray>
    length         float64 40.0
Dimensions without coordinates: nlat, nlon
Data variables:
    SST            (time, member_id, nlat, nlon) float64 dask.array<chunksize=(180, 1, 93, 320), meta=np.ndarray>
    Chl_surf       (time, member_id, nlat, nlon) float64 dask.array<chunksize=(180, 1, 93, 320), meta=np.ndarray>
    KGP            (time, member_id, nlat, nlon) float64 dask.array<chunksize=(180, 1, 93, 320), meta=np.ndarray>
    NPP            (time, member_id, nlat, nlon) float64 dask.array<chunksize=(180, 1, 93, 320), meta=np.ndarray>
    Jint_100m_DIC  (time, member_id, nlat, nlon) float64 dask.array<chunksize=(180, 1, 93, 320), meta=np.ndarray>

In [7]:
from ncar_jobqueue import NCARCluster
from dask.distributed import Client
try:
    cluster
    client
except:
    cluster = NCARCluster(project='UCUC0006')
    cluster.scale(35)
    client = Client(cluster) # Connect this local process to remote workers
client

Client Scheduler: tcp://10.12.205.11:46173 Dashboard: https://jupyterhub.ucar.edu/dav/user/zephyrs/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [8]:
ds_toe = xr.Dataset()

In [9]:
for v in ['SST']:
    # stack lateral dims
    da = ds_djf[v].stack(non_time_dims=[d for d in ds_djf[v].dims if d != 'time'])

    # initialize temporary variables
    da_inflection_stack = da.isel(time=0, drop=True)   
    da_toe_stack = da.isel(time=0, drop=True)

    # loop over non time dims; compute ToE
    toe_results = []
    for i in range(len(da.non_time_dims)):
        toe_results.append(dask.delayed(assess_toe)(da[:, i]))  
     
    results = dask.compute(*toe_results)
    da_inflection_stack.data = np.array([r[0] for r in results]).astype(da.time.dtype)
    da_toe_stack.data = np.array([r[1] for r in results]).astype(da.time.dtype)
    
    # assign to output dataset
    ds_toe[f'{v}_inflect'] = da_inflection_stack.unstack('non_time_dims')
    ds_toe[v] = da_toe_stack.unstack('non_time_dims')
    


KeyboardInterrupt: 

In [ ]:
ds_toe = ds_toe.where(ds_toe.KMT > 0)

### Re-write to include trend sign

In [ ]:
%%time
ds_trend_sign = xr.where(
    ds_djf.sel(time=slice(2091, 2100)).mean(['time', 'member_id']) >= ds_ctrl_mean, 1., -1
).compute()
ds_trend_sign   

In [ ]:
def assess_toe(da):
    """ compute toe on the basis of henson et al 2017"""
    # compute anomalies
    da_ref = da.sel(time=slice(1920, 1950)).mean(['time'])
    anom = da - da_ref
    
    # find inflection point
    time = da.time.values
    anom_cumtrap = integrate.cumtrapz(anom.values, time, initial=0)
    # account for sign of trend 
    trend_sign = xr.where(
        anom_cumtrap.sel(time=slice(2091, 2100)) >= anom, 1., -1).compute()
    
    for l in range(len(time)):
        if all(anom_cumtrap[l:] > 0): #decline
            ndx = l
            break #once it finds the year where the trend declines, break, then...
    
    # compute polyfit line of climate change trend
    beta = np.polyfit(time[ndx:], anom[ndx:], 1)
    
    return time[ndx], beta[0]
